In [25]:
from onsset import *
from IPython.display import display, Markdown, HTML
%matplotlib inline

2021-10-28 16:24:05,960		Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [26]:
### The cost in USD/kW to for power generation capacity additions to the grid
grid_power_plants_capital_cost = {2025: 2699, 
                                  2030: 3196, 
                                  2040: 2364, 
                                  2050: 2285, 
                                  2060: 2371, 
                                  2070: 2242}

### Stand-alone Diesel capital cost (USD/kW) as expected in the years of the analysis
sa_diesel_capital_cost = {2025: 999999, 
                          2030: 999999, 
                          2040: 999999, 
                          2050: 999999, 
                          2060: 999999, 
                          2070: 999999}

### Mini-grid Diesel capital cost (USD/kW) as expected in the years of the analysis
mg_diesel_capital_cost = {2025: 721, 
                          2030: 721, 
                          2040: 721, 
                          2050: 721, 
                          2060: 721, 
                          2070: 721}

### Mini-grid PV capital cost (USD/kW) as expected in the years of the analysis
mg_pv_capital_cost = {2025: 2829, 
                      2030: 2540, 
                      2040: 2252, 
                      2050: 1977, 
                      2060: 1703, 
                      2070: 1428}

### Mini-grid Wind capital cost (USD/kW) as expected in the years of the analysis
mg_wind_capital_cost = {2025: 4356, 
                        2030: 4285, 
                        2040: 4213, 
                        2050: 4117, 
                        2060: 4021, 
                        2070: 3926}

### Mini-grid Hydro capital cost (USD/kW) as expected in the years of the analysis
mg_hydro_capital_cost = {2025: 3000, 
                         2030: 3000, 
                         2040: 3000, 
                         2050: 3000, 
                         2060: 3000, 
                         2070: 3000}          

### Stand-alone PV capital cost (USD/kW) for household systems under 20 W
sa_pv_capital_cost_1 = {2025: 9067, 
                        2030: 8143, 
                        2040: 7218, 
                        2050: 6338, 
                        2060: 5458, 
                        2070: 4577}
### Stand-alone PV capital cost (USD/kW) for household systems between 21-50 W
sa_pv_capital_cost_2 = {2025: 8487, 
                        2030: 7621, 
                        2040: 6756, 
                        2050: 5932, 
                        2060: 5108, 
                        2070: 4285}
### Stand-alone PV capital cost (USD/kW) for household systems between 51-100 W
sa_pv_capital_cost_3 = {2025: 6165, 
                        2030: 5537, 
                        2040: 4908, 
                        2050: 4310, 
                        2060: 3711, 
                        2070: 3113}
### Stand-alone PV capital cost (USD/kW) for household systems between 101-1000 W
sa_pv_capital_cost_4 = {2025: 4316, 
                        2030: 3876, 
                        2040: 3436, 
                        2050: 3017, 
                        2060: 2598, 
                        2070: 2179}
### Stand-alone PV capital cost (USD/kW) for household systems over 1 kW
sa_pv_capital_cost_5 = {2025: 6710, 
                        2030: 6026, 
                        2040: 5342, 
                        2050: 4690, 
                        2060: 4039, 
                        2070: 3387}    

In [27]:
path = r'C:\Users\asahl\Box Sync\PhD\Paper 2 Ethiopia\Model_runs\BigBusiness\LowHangingFruit\Compat_0\BigBusiness_Compat_0_LowHangingFruit_Results.csv'

In [28]:
def rural_summaries(path):
    df_in = pd.read_csv(path)
    df_in['PVType2018'] = 1
    df_in['NewConnections2018'] = 0
    df_in['NewCapacity2018'] = 0
    df_in['InvestmentCost2018'] = 0
    df_in['TransmissionInvestmentCost2018'] = 0
    df_in['TotalEnergyPerCell2018'] = df_in['ElecPopCalib'] * df_in['ResidentialDemandTierCustom2018'] * df_in[SET_ELEC_CURRENT]
    SET_TRANSMISSION_INV = 'TransmissionInvestmentCost'

    elements = ["1.Population", "2.New_Connections", "3.Capacity", "4.Investment", "5. Demand", "6. Transmission summaries", "7. Distribution summaries", "8. Capacity factor"]
    techs = ["Grid", "MG_Diesel", "MG_PV", "MG_Wind", "MG_Hydro", "SA_PV_1", "SA_PV_2", "SA_PV_3", "SA_PV_4", "SA_PV_5"]
    years = [2018, 2025, 2030, 2040, 2050, 2060, 2070]
    tech_codes = [1, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    tech_costs = {1: grid_power_plants_capital_cost, 4: mg_diesel_capital_cost, 5: mg_pv_capital_cost, 6: mg_wind_capital_cost, 
                  7: mg_hydro_capital_cost, 8: sa_pv_capital_cost_1, 9: sa_pv_capital_cost_2, 10: sa_pv_capital_cost_3,
                  11: sa_pv_capital_cost_4, 12: sa_pv_capital_cost_5}

    sumtechs = []
    for element in elements:
        for tech in techs:
            sumtechs.append(element + "_" + tech)

    total_rows = len(sumtechs)

    df_rural = df_in.loc[df_in['IsUrban'] < 2]
    rural_sumtechs = pd.DataFrame(columns=years)

    for row in range(0, total_rows):
        rural_sumtechs.loc[sumtechs[row]] = 0

    i = 0

    # Adding pop summary (Million ppl)
    for code in tech_codes:
        for year in years:
            rural_sumtechs[year][sumtechs[i]] = sum(df_rural.loc[df_rural[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_POP + "{}".format(year)])/1000000
        i += 1

    # Adding connections summary (Million ppl)
    for code in tech_codes:
        for year in years:
            rural_sumtechs[year][sumtechs[i]] = sum(df_rural.loc[df_rural[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_NEW_CONNECTIONS + "{}".format(year)])/1000000
        i += 1

    # Adding capacity summaries (MW)
    for code in tech_codes:
        for year in years:
            rural_sumtechs[year][sumtechs[i]] = sum(df_rural.loc[df_rural[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_NEW_CAPACITY + "{}".format(year)])/1000
        i += 1

    # Adding investment summaries (Million USD)
    for code in tech_codes:
        for year in years:
            rural_sumtechs[year][sumtechs[i]] = sum(df_rural.loc[df_rural[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_INVESTMENT_COST + "{}".format(year)])/1000000
        i += 1

    # Adding demand summaries (GWh)
    for code in tech_codes:
        for year in years:
            rural_sumtechs[year][sumtechs[i]] = sum(df_rural.loc[df_rural[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_TOTAL_ENERGY_PER_CELL + "{}".format(year)])/1000000
        i += 1


    for code in tech_codes:
        for year in years:
            if code == 5 or code > 7:
                if rural_sumtechs[year][sumtechs[i-40]] > 0:
                    rural_sumtechs[year][sumtechs[i]] = df_rural.loc[df_rural[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_GHI].mean()/8760
            elif code == 6:
                if rural_sumtechs[year][sumtechs[i - 40]] > 0:
                    rural_sumtechs[year][sumtechs[i]] = df_rural.loc[df_rural[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_WINDCF].mean()
        i += 1
    
    return rural_sumtechs

In [30]:
scenario_folders = ['LCOE', 'Pop', 'TravelTime', 'LowHangingFruit']
options = ['0', '1', '2']

In [33]:
for scenario in scenario_folders:
    for option in options:
        path = os.path.join(r'C:\Users\asahl\Box Sync\PhD\Paper 2 Ethiopia\Model_runs\BigBusiness', scenario, 'Compat_{}'.format(option), 'BigBusiness_Compat_{}_{}_Results.csv'.format(option, scenario))
        summary = rural_summaries(path)
        path_out = os.path.join(r'C:\Users\asahl\Box Sync\PhD\Paper 2 Ethiopia\Model_runs\BigBusiness', scenario, 'Compat_{}'.format(option), 'BigBusiness_Compat_{}_{}_RuralResults.csv'.format(option, scenario))
        summary.to_csv(path_out, index=True)
        